# Plot contour line Y band resolution vs (dPWV,PWV)

- author Sylvie Dagoret-Campagne
- affiliation IJCLab
- creation date : 2025/11/18
- last update : 2025/11/18
- last update : 2025/11/20

- Read a single file

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.cm as cm
from matplotlib.lines import Line2D


from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd
import h5py

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [ ]:
from scipy.interpolate import splprep, splev

## Functions

In [ ]:
def resample_curve(x, y, spacing):
    """Return a new curve with points spaced approximately 'spacing' apart."""
    # parametrize original curve
    tck, u = splprep([x, y], s=0)
    
    # coordinates length
    pts = np.vstack([x, y]).T
    d = np.sqrt(((pts[1:] - pts[:-1])**2).sum(axis=1))
    s_total = d.sum()
    
    # new parameterization
    u_new = np.linspace(0, 1, int(s_total/spacing))
    x_new, y_new = splev(u_new, tck)
    return np.array([x_new, y_new]).T

## Configuration

In [ ]:
!ls data/mag_vs_zdpwvzpwv/results 

In [ ]:
data_path = "data/mag_vs_zdpwvzpwv/results"

In [ ]:
all_airmass = np.array([1.,1.2,1.5,2.0]) 

In [ ]:
Delta_PWV = 0.5
Delta_PWV_num = int(Delta_PWV*100.)
#all_PWV = np.arange(Delta_PWV,20.,Delta_PWV)
#print("PWV values",all_PWV) 

In [ ]:
#all_DPWV = np.arange(2.,0.,-0.01)
Delta_dPWV = 0.02
Delta_dPWV_num = int(Delta_dPWV*1000.)
#all_DPWV = np.arange(Delta_dPWV,1.,Delta_dPWV)
#print(all_DPWV)

## Read file

In [ ]:
idx_am = 3

In [ ]:
am = all_airmass[idx_am] 
am_num = int(am*10.)

In [ ]:
filename_input =  f"MagResolutionFromPWVvsPWV_am{am_num}_DPWV{Delta_PWV_num}_DdPWV{Delta_dPWV_num}.h5"
fullfilename_input = os.path.join(data_path,filename_input)

In [ ]:
with h5py.File(fullfilename_input, "r") as f:
    print(list(f.keys()))

In [ ]:
with h5py.File(fullfilename_input, "r") as f:
    zPWV   = f["zPWV"][:]
    zdPWV  = f["zdPWV"][:]
    tresY  = f["tresY"][:]
    tresZY = f["tresZY"][:]

## Plot

In [ ]:
x = zPWV               # shape (nx,)
y = zdPWV           # shape (ny,)
Z = tresY              # shape (ny, nx)

X, Y = np.meshgrid(x, y)

plt.figure(figsize=(10,8))
pcm = plt.pcolormesh(X, Y, Z, shading='auto',cmap='magma')   # shading='auto' avoids dimension issues

cs = plt.contour(X, Y, Z, levels= np.array([0.5,1.,5.,10.,15.,20.,25.,30.]) ,colors='w', linewidths=1.0)
plt.clabel(cs, inline=True, fontsize=8)

plt.colorbar(pcm, label='$\\sigma(m_y)$ (mmag)')
plt.xlabel('airmass  x PWV (mm)')
plt.ylabel('airmass  x ΔPWV (mm)')
plt.title(f'Magnitude resolution (Y band, airmass = {am})')

ax = plt.gca()
ax.minorticks_on()
#ax.set_xticks(np.arange(zPWV.min(), zPWV.max(), 0.1))   # 1 mm steps
#ax.set_yticks(np.arange(all_DPWV.min(), all_DPWV.max()+0.01, -0.1))  # 0.1 mm steps

plt.show()

In [ ]:
plt.figure(figsize=(10,8))

im = plt.imshow(Z, origin='lower',
           extent=[x.min(), x.max(), y.min(), y.max()],
           aspect='auto',cmap='magma')

cs = plt.contour(X, Y, Z, levels= np.array([0.5,1.,5.,10.,15.,20.,25.,30.]) ,colors='w', linewidths=2.0)
plt.clabel(cs, inline=True, fontsize=8)

plt.colorbar(im,label='$\\sigma(m_y)$ (mmag)')
plt.xlabel(r'airmass $\times$ PWV (mm)')
plt.ylabel(r'airmass $ \times \; \sigma(PWV (mm))$')
plt.title(f'Magnitude resolution (Y band, airmass = {am})')

ax = plt.gca()
ax.minorticks_on()
#ax.set_xticks(np.arange(zPWV.min(), zPWV.max()+1, 1))   # 1 mm steps
#ax.set_yticks(np.arange(all_DPWV.min(), all_DPWV.max()+0.01, -0.1))  # 0.1 mm steps

plt.show()

In [ ]:
plt.figure(figsize=(10,8))

plt.imshow(Z, origin='lower', extent=[x.min(),x.max(),y.min(),y.max()],
           aspect='auto', cmap='magma',alpha=0.1)

for level, seglist in zip(cs.levels, cs.allsegs):
    for curve in seglist:
        plt.plot(curve[:,0], curve[:,1], '-', linewidth=1.0, label=f"Level {level}")
plt.xlabel('airmass x PWV (mm)')
plt.ylabel('airmass x ΔPWV (mm)')
plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
plt.title(f'Magnitude resolution (Y band, airmass = {am})')

In [ ]:
plt.figure(figsize=(10,8))


for level, seglist in zip(cs.levels, cs.allsegs):
    for curve in seglist:
        plt.plot(curve[:,0], curve[:,1]/level, '-', linewidth=1.0, label=f"Level {level}")
plt.xlabel('airmass x PWV (mm)')
plt.ylabel('airmass x ΔPWV (mm)')
plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
plt.title(f'Magnitude resolution (Y band, airmass = {am})')

## Smooth the contour lines

In [ ]:
spacing = 0.1  # choose your spacing in same units as X,Y grid

resampled_contours = {}  # dict[level] = [array_of_points_per_curve]

for level, seglist in zip(cs.levels, cs.allsegs):
    resampled_contours[level] = []
    for curve in seglist:
        xc = curve[:,0]
        yc = curve[:,1]
        if len(xc) < 4:   # or < k+1
            continue
        new_curve = resample_curve(xc, yc, spacing)
        resampled_contours[level].append(new_curve)


In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(Z, origin='lower', extent=[x.min(),x.max(),y.min(),y.max()],
           aspect='auto', cmap='magma',alpha=0.1)

for level, curves in resampled_contours.items():
    for c in curves:
        plt.plot(c[:,0], c[:,1], '-', linewidth=0.6,label=f"Level {level}")

plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
plt.title("Resampled contour curves")
plt.show()


## Colored

In [ ]:
levels = cs.levels
cmap = plt.cm.coolwarm
norm = colors.Normalize(vmin=min(levels), vmax=max(levels))

In [ ]:
plt.figure(figsize=(9,8))

# --- Background image ---
plt.imshow(Z, origin='lower',
           extent=[x.min(),x.max(),y.min(),y.max()],
           aspect='auto', cmap='magma', alpha=0.1)

# --- Colormap for contour lines ---
levels = cs.levels
cmap = plt.cm.coolwarm            # dark blue → red
norm = colors.Normalize(vmin=min(levels), vmax=max(levels))

# --- Plot contour lines manually ---
for level, seglist in zip(cs.levels, cs.allsegs):
    color = cmap(norm(level))
    for curve in seglist:
        plt.plot(curve[:,0], curve[:,1],
                 '-', linewidth=3.0, color=color)

# --- Legend: 1 entry per level ---
legend_elements = [
    Line2D([0], [0], color=cmap(norm(level)), lw=2,
            label=r"$\sigma_(m_Y)$ = " + f"{level:g} mmag")
    for level in cs.levels
]
plt.legend(handles=legend_elements,
           loc='center left', bbox_to_anchor=(0.75, 0.5),
           title="Contour levels")

plt.xlabel('airmass × PWV (mm)')
plt.ylabel(r'airmass × $\sigma(PWV (mm))$')
plt.title(f'$m_Y$ resolution (Y band, airmass = {am})')

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(9,8))

# --- Background image ---
#plt.imshow(Z, origin='lower',
#           extent=[x.min(),x.max(),y.min(),y.max()],
#           aspect='auto', cmap='magma', alpha=0.1)

# --- Colormap for contour lines ---
levels = cs.levels
cmap = plt.cm.coolwarm            # dark blue → red
norm = colors.Normalize(vmin=min(levels), vmax=max(levels))

# --- Plot contour lines manually ---
for level, seglist in zip(cs.levels, cs.allsegs):
    color = cmap(norm(level))
    for curve in seglist:
        plt.plot(curve[:,0], curve[:,1]/level,'-', linewidth=2.0, color=color)

# --- Legend: 1 entry per level ---
legend_elements = [
    Line2D([0], [0], color=cmap(norm(level)), lw=2,
           label=r"$\sigma_(m_Y)$ = " + f"{level:g} mmag")
    for level in cs.levels
]
plt.legend(handles=legend_elements,
           loc='center left', bbox_to_anchor=(0.03, 0.8),
           title="Contour levels")

plt.xlabel('airmass × PWV (mm)')
plt.ylabel(r'airmass × $\frac{\sigma (PWV(mm))}{\sigma(m_Y)(mmag)}$')
plt.title(r'$\sigma(m_Y)$-normalized PWV resolution vs PWV')
plt.xlim(0.,35)
plt.ylim(0.,.2)

plt.tight_layout()
plt.show()


## Does the fit

#### Gather points to fit

In [ ]:
# --- Plot contour lines manually ---
for level, seglist in zip(cs.levels, cs.allsegs):
    color = cmap(norm(level))
    for curve in seglist:
        Xpoints = curve[:,0]
        Ypoints = curve[:,1]/level


### Function to fit

- $X = z \cdot PWV$

$$ dX = \frac{1}{\alpha} \frac{\ln(10)}{2.5} \frac{dm_Y}{K_Y}\cdot X^{(1-\alpha)} = \beta \cdot dm_Y \cdot X^{1-\alpha}$$

$$\frac{dX}{X} = \frac{\ln(10)}{2.5} \frac{1}{\alpha K_Y} \frac{dm_Y}{X^\alpha} = \beta \cdot \frac{dm_Y}{X^\alpha}$$

$$
\beta = \frac{1}{\alpha K_Y}
$$

In [ ]:
def func01(x,alpha,beta):
    """
    """
    return np.power(x,1-alpha)*beta

In [ ]:
def func1(x,alpha,beta,dm):
    """
    """
    return np.power(x,1-alpha)*beta*dm

In [ ]:
def func02(x,alpha,beta):
    """
    """
    return np.power(x,-alpha)*beta

In [ ]:
assert False

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from scipy.optimize import curve_fit
from matplotlib.cm import get_cmap

# --- Style publication ---
plt.style.use("seaborn-v0_8-whitegrid")
rcParams.update({
    "font.family": "serif",
    "font.size": 14,
    "axes.labelsize": 16,
    "axes.titlesize": 17,
    "legend.fontsize": 13,
    "xtick.labelsize": 13,
    "ytick.labelsize": 13,
    "axes.linewidth": 1.2,
    "lines.linewidth": 2.2,
    "lines.markersize": 6,
    "figure.dpi": 200
})

fig, ax = plt.subplots(figsize=(8.5, 6.5), layout="constrained")

# --- Palette harmonieuse ---
#palette = get_cmap("viridis")

palette_left = get_cmap("viridis")
palette_right = get_cmap("plasma")

color_map = {
    '10mmag': (palette_left(0.15), "$\\sigma (m_Y) = 10$ mmag"),
    '05mmag': (palette_left(0.45), "$\\sigma (m_Y) = 5$ mmag"),
    '01mmag': (palette_left(0.75), "$\\sigma (m_Y) = 1$ mmag"),
}

ls_labels = {
    '-.': "$Z = 1.0$",
    '-': "$Z = 1.2$",
    '--': "$Z = 1.5$",
    ':': "$Z = 2.0$",
}

magname_to_mag = {'10mmag': 10., '05mmag': 5., '01mmag': 1.}

# --- Courbes principales (axe gauche) ---
for key, (color, label) in color_map.items():

    dm = magname_to_mag[key]
    XX = []
    YY = []

    # boucle sur airmass
    for idx_am in range(NAM):
        ls = airmass_sel_ls[idx_am]
        data = all_data_sigma[idx_am]

        X = np.array(data[f'pwv{key}']) * airmass_sel[idx_am]
        Y = np.array(data[f'dpwv{key}']) * airmass_sel[idx_am]

        X, Y = X[:-4], Y[:-4]
        Y = Y / dm

        ax.plot(X, Y, marker="o", color=color, ls=ls, alpha=0.9)

        XX.append(X)
        YY.append(Y)


XX= np.hstack(XX)    
YY = np.hstack(YY)

def func_to_fit(x, alpha, beta):
    return func01(x, alpha, beta)

popt, pcov = curve_fit(func_to_fit, XX, YY, p0=[0.5, 0.1])
xfit = np.linspace(XX.min(), XX.max(), 200)
ax.plot(xfit, func_to_fit(xfit, *popt), "-", lw=3, color="black", alpha=0.7)

perr = np.sqrt(np.diag(pcov))
fit_text = (
    #f"$\\alpha = {popt[0]:.3f} \\pm {perr[0]:.3f}$\n"
    #f"$\\beta = {popt[1]:.3f} \\pm {perr[1]:.3f}$"
    f"$\\alpha = {popt[0]:.3f}$\n"
    f"$\\beta = {popt[1]:.3f}$"
)
ax.text(
    0.8, 0.55, fit_text,
    transform=ax.transAxes,
    fontsize=13, va="top", ha="left", color="black",
    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", edgecolor="gray", alpha=0.9)
)

# --- Axe de droite (ax2) ---
ax2 = ax.twinx()
#color_map2 = {
#    '10mmag': (palette(0.15), "$\\Delta m(Y) = 10$ mmag"),
#    '05mmag': (palette(0.45), "$\\Delta m(Y) = 5$ mmag"),
#    '01mmag': (palette(0.75), "$\\Delta m(Y) = 1$ mmag"),
#}

color_map2 = {
    '10mmag': (palette_right(0.15), "$\\sigma (m_Y) = 10$ mmag"),
    '05mmag': (palette_right(0.45), "$\\sigma (m_Y) = 5$ mmag"),
    '01mmag': (palette_right(0.75), "$\\sigma (m_Y) = 1$ mmag"),
}

for key, (color, label) in color_map2.items():


    dm = magname_to_mag[key]
    XX2 = []
    YY2 = []


    
    for idx_am in range(NAM):
        ls = airmass_sel_ls[idx_am]
        data = all_data_sigma[idx_am]

        X = np.array(data[f'pwv{key}']) * airmass_sel[idx_am]
        Y = np.array(data[f'dpwv{key}']) * airmass_sel[idx_am]
        X, Y = X[:-4], Y[:-4]

        Y = (Y / X) / dm
        ax2.plot(X, Y, marker="s", color=color, ls=ls, alpha=0.6)


        XX2.append(X)
        YY2.append(Y)
       
XX2 = np.hstack(XX2)    
YY2 = np.hstack(YY2)


def func_to_fit2(x, alpha, beta):
    return func02(x, alpha, beta)

popt2, pcov2 = curve_fit(func_to_fit2, XX2, YY2, p0=[0.5, 0.1])
xfit2 = np.linspace(XX2.min(), XX2.max(), 200)
ax2.plot(xfit2, func_to_fit2(xfit2, *popt2), "-", lw=3, color="darkred", alpha=0.8)

# --- Axes, titres et légendes ---
ax.set_xlabel("$airmass \\times PWV$ (mm)")
ax.set_ylabel("$\\frac{airmass \\times \\sigma(PWV/1mm)}{\\sigma(m)/1mmag)}$", color="black")
ax2.set_ylabel("$\\frac{\\sigma(PWV)}{PWV} / (\\sigma(m)/1mmag)$", color="darkgreen")
ax.set_title("PWV resolution vs PWV for different magnitude resolutions (Y band)")

# Grille fine
ax.grid(True, which="major", ls="--", lw=0.6, alpha=0.4)
ax.minorticks_on()
ax.tick_params(which="minor", length=4, width=1)

# Légendes séparées
handles_color = [plt.Line2D([], [], color=c, lw=2, label=l) for c, l in color_map.values()]
handles_ls = [plt.Line2D([], [], color="gray", ls=ls, lw=2, label=lab) for ls, lab in ls_labels.items()]

leg1 = ax.legend(handles=handles_color, loc="upper left", title="Magnitude resolution")
ax.add_artist(leg1)
ax.legend(handles=handles_ls, loc="lower right", title="Airmass")
ax.tick_params(axis='y', colors='darkgreen')
ax.spines['left'].set_color('darkgreen')
ax.yaxis.label.set_color("darkgreen")

# Harmonisation des axes
ax2.tick_params(axis='y', colors='darkred')
ax2.spines['right'].set_color('darkred')
ax2.yaxis.label.set_color("darkred")
ax2.set_ylim(0., 0.05)
ax.minorticks_on()
ax2.grid(False)

#fig.text(0.5, 0.02,
#         "Left axis: $airmass \\times \\sigma(PWV/1mmag)/(\\sigma(m)/1mmag)$   |   "
#         "Right axis: $\\sigma(PWV)/PWV / (\\sigma(m)/1mmag)$",
#         ha="center", fontsize=12, color="dimgray")

plt.tight_layout()


figname =f"{pathfigs}/{prefix}_fit_finalcombined_amdeltapwv_vs_amPWV_linvscale-divdeltam"+figtype
plt.savefig(figname)

plt.show()
